# Exploración

In [1]:
!ls

clase_11b_bayes_word2vec.pdf  requirements.txt		      tarea3.ipynb
data			      Reuters21578-Apte-90Cat.tar.gz  tarea3.pdf


In [2]:
!ls data

test  training


## Training

In [3]:
!ls data/training/

acq	     cpu	    income	  naphtha     rape-oil	       sun-oil
alum	     crude	    instal-debt   nat-gas     rapeseed	       sunseed
barley	     dfl	    interest	  nickel      reserves	       tea
bop	     dlr	    ipi		  nkr	      retail	       tin
carcass      dmk	    iron-steel	  nzdlr       rice	       trade
castor-oil   earn	    jet		  oat	      rubber	       unknown
cocoa	     fuel	    jobs	  oilseed     rye	       veg-oil
coconut      gas	    l-cattle	  orange      ship	       wheat
coconut-oil  gnp	    lead	  palladium   silver	       wpi
coffee	     gold	    lei		  palmkernel  sorghum	       yen
copper	     grain	    lin-oil	  palm-oil    soybean	       zinc
copra-cake   groundnut	    livestock	  pet-chem    soy-meal
corn	     groundnut-oil  lumber	  platinum    soy-oil
cotton	     heat	    meal-feed	  potato      strategic-metal
cotton-oil   hog	    money-fx	  propane     sugar
cpi	     housing	    money-supply  rand	      sun-meal


## Dataset: Acquisition 

In [4]:
!ls data/training/acq 

0000005  0001151  0002088  0003205  0004272  0005419  0006378  0007598	0008473
0000007  0001152  0002090  0003210  0004284  0005428  0006384  0007601	0008483
0000027  0001160  0002093  0003211  0004300  0005436  0006388  0007617	0008493
0000028  0001166  0002095  0003237  0004306  0005441  0006393  0007618	0008497
0000044  0001169  0002102  0003241  0004311  0005442  0006394  0007627	0008510
0000065  0001171  0002107  0003260  0004319  0005444  0006396  0007630	0008513
0000078  0001183  0002112  0003284  0004323  0005446  0006425  0007637	0008514
0000087  0001204  0002117  0003294  0004329  0005448  0006428  0007638	0008527
0000090  0001256  0002120  0003299  0004330  0005449  0006431  0007641	0008541
0000095  0001257  0002122  0003301  0004345  0005457  0006446  0007642	0008545
0000096  0001260  0002123  0003310  0004363  0005460  0006474  0007653	0008549
0000112  0001262  0002124  0003312  0004364  0005461  0006489  0007656	0008550
0000114  0001271  0002125  0003332  0004365  0005462

## Algún archivo

In [5]:
!cat data/training/acq/0000005

## Dataset: CPU

In [6]:
!ls data/training/cpu

0003746  0003797  0003816


## Buscar más Frecuentes

In [7]:
import os
trainingFiles = []
for root, dirs, files in os.walk("data/training"):
    for file in files:
        trainingFiles.append([root, file])
        
trainingFiles[0:10]

[['data/training/naphtha', '0004556'],
 ['data/training/naphtha', '0005167'],
 ['data/training/soybean', '0004358'],
 ['data/training/soybean', '0008468'],
 ['data/training/soybean', '0000885'],
 ['data/training/soybean', '0002152'],
 ['data/training/soybean', '0001505'],
 ['data/training/soybean', '0001131'],
 ['data/training/soybean', '0005250'],
 ['data/training/soybean', '0006770']]

## Agrupar

In [8]:
import pandas as pd

In [9]:
labels = ['root', 'file']
df = pd.DataFrame.from_records(trainingFiles, columns=labels)

In [10]:
df[0:10]

,root,file
0,data/training/naphtha,0004556
1,data/training/naphtha,0005167
2,data/training/soybean,0004358
3,data/training/soybean,0008468
4,data/training/soybean,0000885
5,data/training/soybean,0002152
6,data/training/soybean,0001505
7,data/training/soybean,0001131
8,data/training/soybean,0005250
9,data/training/soybean,0006770


In [11]:
df.describe()

,root,file
count,11413,11413
unique,91,9598
top,data/training/earn,0000167
freq,2877,15


## GroupBy

In [12]:
df.groupby('root').describe()

file                     
                              count unique      top freq
root                                                    
data/training/acq              1650   1650  0008228    1
data/training/alum               35     35  0006562    1
data/training/barley             37     37  0005296    1
data/training/bop                75     75  0007232    1
data/training/carcass            50     50  0003480    1
data/training/castor-oil          1      1  0007237    1
data/training/cocoa              55     55  0006724    1
data/training/coconut             4      4  0007537    1
data/training/coconut-oil         4      4  0006017    1
data/training/coffee            111    111  0007110    1
data/training/copper             47     47  0004030    1
data/training/copra-cake          2      2  0000167    1
data/training/corn              181    181  0002996    1
data/training/cotton             39     39  0001012    1
data/training/cotton-oil          1      1  0004459    1
data/training/cpi                69     69  0008538    1
data/training/cpu                 3      3  0003797    1
data/training/crude             389    389  0008210    1
data/training/dfl                 2      2  0007273    1
data/training/dlr               131    131  0007511    1
data/training/dmk                10     10  0002014    1
data/training/earn             2877   2877  0002387    1
data/training/fuel               13     13  0001919    1
data/training/gas                37     37  0008210    1
data/training/gnp               101    101  0007956    1
data/training/gold               94     94  0001527    1
data/training/grain             433    433  0003501    1
data/training/groundnut           5      5  0001490    1
data/training/groundnut-oil       1      1  0000222    1
data/training/heat               14     14  0008363    1
...                             ...    ...      ...  ...
data/training/potato              3      3  0001242    1
data/training/propane             3      3  0000493    1
data/training/rand                2      2  0006570    1
data/training/rape-oil            5      5  0005713    1
data/training/rapeseed           18     18  0003141    1
data/training/reserves           55     55  0004016    1
data/training/retail             23     23  0003643    1
data/training/rice               35     35  0008181    1
data/training/rubber             37     37  0007882    1
data/training/rye                 1      1  0000066    1
data/training/ship              197    197  0007474    1
data/training/silver             21     21  0008689    1
data/training/sorghum            24     24  0003919    1
data/training/soy-meal           13     13  0000045    1
data/training/soy-oil            14     14  0009255    1
data/training/soybean            78     78  0000885    1
data/training/strategic-metal    16     16  0001906    1
data/training/sugar             126    126  0008590    1
data/training/sun-meal            1      1  0008798    1
data/training/sun-oil             5      5  0002727    1
data/training/sunseed            11     11  0008613    1
data/training/tea                 9      9  0007320    1
data/training/tin                18     18  0005895    1
data/training/trade             369    369  0004004    1
data/training/unknown          1830   1830  0000345    1
data/training/veg-oil            87     87  0000226    1
data/training/wheat             212    212  0002815    1
data/training/wpi                19     19  0003218    1
data/training/yen                45     45  0007586    1
data/training/zinc               21     21  0004895    1

[91 rows x 4 columns]

## Ordernar y sacar los 10
Los 11 en realidad, porque hay una categoría unkwown

In [13]:
largest = df.groupby('root').count().nlargest(11, columns="file")
largest

,file
root,
data/training/earn,2877
data/training/unknown,1830
data/training/acq,1650
data/training/money-fx,538
data/training/grain,433
data/training/crude,389
data/training/trade,369
data/training/interest,347
data/training/wheat,212


In [14]:
largest.describe()

,file
count,11.000000
mean,820.272727
std,891.546195
min,181.000000
25%,279.500000
50%,389.000000
75%,1094.000000
max,2877.000000


## Iterate trough values:

In [15]:
for row in df.iterrows():
    print(row[0], row[1]["root"], row[1]["file"])
    break

0 data/training/naphtha 0004556


In [16]:
for row in largest.iterrows():
    print(row[0], row[1]["file"])

data/training/earn 2877
data/training/unknown 1830
data/training/acq 1650
data/training/money-fx 538
data/training/grain 433
data/training/crude 389
data/training/trade 369
data/training/interest 347
data/training/wheat 212
data/training/ship 197
data/training/corn 181


## Refactor

In [17]:
path = "data/training"
def getAllFiles(path):
    allFiles = []
    for root, dirs, files in os.walk(path):
        for file in files:
            allFiles.append([root, file])
            
    return allFiles

In [18]:
def getTop10Categories(files):
    labels = ['root', 'file']
    df = pd.DataFrame.from_records(files, columns=labels)
    largest = df.groupby('root').count().nlargest(11, columns="file")
    return [[row[0], row[1]["file"]] for row in largest.iterrows() if "unknown" not in row[0]]

In [19]:
def getFilesInFolders(folderList):
    allFiles = []
    for folder in folderList:
        for root, dirs, files in os.walk(folder):
            for file in files:
                allFiles.append([root, file])
            
    return allFiles

In [20]:
def concatFiles(allFiles):
    return [str(r[0])+ "/" + str(r[1]) for r in allFiles]

## Preview

### All Files

In [21]:
trainingFiles = getAllFiles("data/training")
print(len(trainingFiles))
print(trainingFiles[0])

11413
['data/training/naphtha', '0004556']


### Top 10 Categories

In [22]:
top10 = getTop10Categories(trainingFiles)
print(len(top10))
print(top10[0])

10
['data/training/earn', 2877]


### Top 10 Folders

In [23]:
top10Folders = [f[0] for f in top10]
top10Folders

['data/training/earn',
 'data/training/acq',
 'data/training/money-fx',
 'data/training/grain',
 'data/training/crude',
 'data/training/trade',
 'data/training/interest',
 'data/training/wheat',
 'data/training/ship',
 'data/training/corn']

### Top 10 Files

In [24]:
top10Files = getFilesInFolders(top10Folders)
print(len(top10Files))
print("\n".join(concatFiles(top10Files)[::500]))

7193
data/training/earn/0006903
data/training/earn/0001638
data/training/earn/0000416
data/training/earn/0003049
data/training/earn/0001883
data/training/earn/0006551
data/training/acq/0004263
data/training/acq/0007759
data/training/acq/0005147
data/training/acq/0004862
data/training/money-fx/0007282
data/training/crude/0005910
data/training/trade/0003619
data/training/interest/0000568
data/training/ship/0009439


### Check 'earn' Length

In [25]:
earnList = [r for r in top10Files if "earn" in r[0]]
len(earnList)

2877

# Eliminar signos de puntuación

In [26]:
testFile = os.path.join(trainingFiles[0][0], trainingFiles[0][1])
testFile

'data/training/naphtha/0004556'

In [27]:
testFileOpen = open(testFile, "r")
testText = testFileOpen.read()
testFileOpen.close()
print(testText)



PAKISTAN-SWEDISH GOODS EXCHANGE AGREED

    ISLAMABAD, March 18 - Pakistan and Sweden have signed a
commodity exchange agreement for 88 mln dlrs each way, the
Pakistan government announced.
    Pakistan's exports under the agreement will include raw
cotton, cotton products, cotton textiles, steel products,
molasses, naphtha and fresh and dried fruits.
    Swedish exports to Pakistan will include medical and
laboratory equipment, electrical telecommunication equipment,
diesel engine spares, mining and security equipment,
road-building and construction machinery, fertilisers and palm
oil.




## Fuerza bruta

In [28]:
puntuacion = [",", ".", ";", ":", "<", ">", "-"]

In [29]:
sinPuntos = testText

for punto in puntuacion:
    sinPuntos = sinPuntos.replace(punto, "")
    
print(sinPuntos)



PAKISTANSWEDISH GOODS EXCHANGE AGREED

    ISLAMABAD March 18  Pakistan and Sweden have signed a
commodity exchange agreement for 88 mln dlrs each way the
Pakistan government announced
    Pakistan's exports under the agreement will include raw
cotton cotton products cotton textiles steel products
molasses naphtha and fresh and dried fruits
    Swedish exports to Pakistan will include medical and
laboratory equipment electrical telecommunication equipment
diesel engine spares mining and security equipment
roadbuilding and construction machinery fertilisers and palm
oil




Problemas de este método:
- Tener todos los signos de puntuación en el arreglo
- Eficiencia

## Expresiones regulares

In [30]:
import re, string

In [31]:
sinPuntos = testText

regex = re.compile('[%s]' % re.escape(string.punctuation))
sinPuntos =  regex.sub('', sinPuntos)
    
print(sinPuntos)



PAKISTANSWEDISH GOODS EXCHANGE AGREED

    ISLAMABAD March 18  Pakistan and Sweden have signed a
commodity exchange agreement for 88 mln dlrs each way the
Pakistan government announced
    Pakistans exports under the agreement will include raw
cotton cotton products cotton textiles steel products
molasses naphtha and fresh and dried fruits
    Swedish exports to Pakistan will include medical and
laboratory equipment electrical telecommunication equipment
diesel engine spares mining and security equipment
roadbuilding and construction machinery fertilisers and palm
oil




## String Translate

In [32]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [33]:
sinPuntos = testText
sinPuntos = sinPuntos.translate(str.maketrans('','', string.punctuation))
print(sinPuntos)



PAKISTANSWEDISH GOODS EXCHANGE AGREED

    ISLAMABAD March 18  Pakistan and Sweden have signed a
commodity exchange agreement for 88 mln dlrs each way the
Pakistan government announced
    Pakistans exports under the agreement will include raw
cotton cotton products cotton textiles steel products
molasses naphtha and fresh and dried fruits
    Swedish exports to Pakistan will include medical and
laboratory equipment electrical telecommunication equipment
diesel engine spares mining and security equipment
roadbuilding and construction machinery fertilisers and palm
oil




## Comparación

[Source](https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python)

```python
replace   : 28.4436721802
regex     : 6.86155414581
translate : 2.12455511093
```

## Eliminar números

In [34]:
sinNumeros = re.sub('\d', '', sinPuntos)
print(sinNumeros)



PAKISTANSWEDISH GOODS EXCHANGE AGREED

    ISLAMABAD March   Pakistan and Sweden have signed a
commodity exchange agreement for  mln dlrs each way the
Pakistan government announced
    Pakistans exports under the agreement will include raw
cotton cotton products cotton textiles steel products
molasses naphtha and fresh and dried fruits
    Swedish exports to Pakistan will include medical and
laboratory equipment electrical telecommunication equipment
diesel engine spares mining and security equipment
roadbuilding and construction machinery fertilisers and palm
oil




# Eliminar Stopwords

Al igual que en el caso anterior, podríamos crear un arreglo y eliminar los stopwords. Una vez más, tendríamos que construir (o [descargar](https://github.com/stopwords-iso/stopwords-en)) un listado de estas.

Para este caso, ocuparemos el [Natural Language Toolkit](http://www.nltk.org/) para hacer esto.

In [35]:
from nltk.corpus import stopwords

In [36]:
stop = set(stopwords.words('english'))
list(stop)[0:10]

['have',
 'into',
 'them',
 'which',
 'through',
 'those',
 'isn',
 'until',
 'that',
 'but']

In [37]:
sinStop = [i for i in sinNumeros.lower().split() if i not in stop]
print(" ".join(sinStop))

pakistanswedish goods exchange agreed islamabad march pakistan sweden signed commodity exchange agreement mln dlrs way pakistan government announced pakistans exports agreement include raw cotton cotton products cotton textiles steel products molasses naphtha fresh dried fruits swedish exports pakistan include medical laboratory equipment electrical telecommunication equipment diesel engine spares mining security equipment roadbuilding construction machinery fertilisers palm oil


# Tokenize

In [38]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/clgadel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
tokenized = [i for i in word_tokenize(sinNumeros.lower()) if i not in stop] 

In [40]:
tokenized[0:20]

['pakistanswedish',
 'goods',
 'exchange',
 'agreed',
 'islamabad',
 'march',
 'pakistan',
 'sweden',
 'signed',
 'commodity',
 'exchange',
 'agreement',
 'mln',
 'dlrs',
 'way',
 'pakistan',
 'government',
 'announced',
 'pakistans',
 'exports']

# Stemming

In [41]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [42]:
stemText = [porter.stem(i.lower()) for i in word_tokenize(sinNumeros) if i.lower() not in stop]

In [43]:
stemText[0:20]

['pakistanswedish',
 'good',
 'exchang',
 'agre',
 'islamabad',
 'march',
 'pakistan',
 'sweden',
 'sign',
 'commod',
 'exchang',
 'agreement',
 'mln',
 'dlr',
 'way',
 'pakistan',
 'govern',
 'announc',
 'pakistan',
 'export']

## Refactor

In [44]:
def processDocument(document):
    sinPuntos = document.translate(str.maketrans('','', string.punctuation))
    sinNumeros = re.sub('\d', '', sinPuntos)
    stemText = [porter.stem(i.lower()) for i in word_tokenize(sinNumeros) if i.lower() not in stop]
    return stemText

In [45]:
def readFile(filename):
    file = open(filename)
    try:
        read = file.read()
    except:
        print(filename)
    file.close()
    return read

In [46]:
def readProcessDocument(documentFilename):
    read = readFile(documentFilename)
    parsed = processDocument(read)
    return parsed

### Test

In [47]:
processDocument(testText)[0:20]

['pakistanswedish',
 'good',
 'exchang',
 'agre',
 'islamabad',
 'march',
 'pakistan',
 'sweden',
 'sign',
 'commod',
 'exchang',
 'agreement',
 'mln',
 'dlr',
 'way',
 'pakistan',
 'govern',
 'announc',
 'pakistan',
 'export']

# Refactor Extract Files

Ahora con la carpeta y una lista de las categorías. Tuve que cambiarlos porque los test podrían no tener los mismos Top 10 de frecuencia.

In [48]:
# Usage: getFilesInFolders("data/training", ["acq", "earn", "money-fx"])
def getFilesInFolders(path, folderList):
    allFiles = []
    for root, dirs, files in os.walk(path):
        if any(word in root for word in folderList):
            for file in files:
                allFiles.append([root, file])
            
    return allFiles

In [49]:
categories = ["acq", "earn", "money-fx", "grain", "crude", "trade", "interest", "ship", "wheat", "corn"]

### Test

In [50]:
getFilesInFolders("data/training", categories)[::500]

[['data/training/grain', '0000574'],
 ['data/training/crude', '0001269'],
 ['data/training/interest', '0001988'],
 ['data/training/acq', '0009235'],
 ['data/training/acq', '0004109'],
 ['data/training/acq', '0001478'],
 ['data/training/trade', '0007488'],
 ['data/training/earn', '0006039'],
 ['data/training/earn', '0005406'],
 ['data/training/earn', '0009218'],
 ['data/training/earn', '0004637'],
 ['data/training/earn', '0001089'],
 ['data/training/earn', '0006397'],
 ['data/training/corn', '0002244'],
 ['data/training/money-fx', '0007525']]

## Extract Files

In [51]:
trainingFiles = getFilesInFolders("data/training", categories)
print(len(trainingFiles))

7193


## Training Set

In [52]:
trainDocuments = [[filename.split("/")[2], readFile(filename)] for filename in concatFiles(trainingFiles[::11])]
print(len(trainDocuments))

654


## Training Data and Target

In [53]:
trainData = [content for file, content in trainDocuments]
trainTarget = [file for file, content in trainDocuments]

## Preview

In [54]:
trainDocumentName = trainTarget[0]
trainDocumentText = trainData[0]
print(trainDocumentName, trainDocumentText)

grain 

ZIMBABWE MAIZE HARVEST LOWER AFTER BUMPER CROPS HARARE, March 3

- Zimbabwe's maize crop in 1986/87 (April/ March) is likely to
be slightly over 1.6 mln tonnes, against 1.83 mln in 1985/86,
Grain Marketing Board figures show.
    Maize exports for 1986/87 to January 31, 1987 totalled
315,000 tonnes, with about a further 40,000 tonnes expected to
be exported in February and March, against 285,000 tonnes in
1985/86. Domestic usage is estimated at 650,000-900,000 tonnes,
depending on how other crops are affected by current poor
rains. Last year's consumption was around 700,000 tonnes.
    Zimbabwe has around two mln stonnes of surplus maize in
storage, accumulated after two years of bumper harvests.





# Count Vectorizer

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

In [56]:
count_vect = CountVectorizer(analyzer = processDocument)

### Fit

In [57]:
count_vect.fit(trainData);

### Fit Vocabulary

In [58]:
print(count_vect.vocabulary_)

{'zimbabw': 5921, 'maiz': 3100, 'harvest': 2314, 'lower': 3048, 'bumper': 707, 'crop': 1226, 'harar': 2295, 'march': 3127, 'april': 285, 'like': 2975, 'slightli': 4798, 'mln': 3323, 'tonn': 5330, 'grain': 2196, 'market': 3138, 'board': 591, 'figur': 1925, 'show': 4730, 'export': 1827, 'januari': 2712, 'total': 5341, 'expect': 1811, 'februari': 1892, 'domest': 1519, 'usag': 5564, 'estim': 1760, 'depend': 1378, 'affect': 97, 'current': 1255, 'poor': 3986, 'rain': 4190, 'last': 2878, 'year': 5886, 'consumpt': 1118, 'around': 325, 'two': 5442, 'stonn': 5003, 'surplu': 5108, 'storag': 5008, 'accumul': 40, 'ccc': 837, 'accept': 30, 'wheat': 5764, 'bid': 556, 'w': 5670, 'africa': 102, 'countri': 1189, 'washington': 5706, 'commod': 1031, 'credit': 1215, 'corport': 1162, 'bonu': 605, 'cover': 1197, 'sale': 4540, 'us': 5562, 'west': 5752, 'african': 103, 'agricultur': 120, 'depart': 1375, 'said': 4530, 'dark': 1292, 'northern': 3559, 'spring': 4917, 'shipment': 4711, 'may': 3168, 'june': 2770, '

### Transform

In [59]:
X_train_counts = count_vect.transform(trainData)

In [78]:
print(X_train_counts.shape)
print(X_train_counts.toarray())

(654, 5932)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


# TF-iDF Transform

In [61]:
from sklearn.feature_extraction.text import TfidfTransformer

## Fit and Transform

In [62]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(654, 5932)

# Naive Bayes

In [63]:
from sklearn.naive_bayes import MultinomialNB

In [64]:
clf = MultinomialNB().fit(X_train_tfidf, trainTarget)

## Test Predict

In [65]:
testFile = "data/test/money-fx/0009664"

In [66]:
testTarget = [testFile.split("/")[2]]
testData = [readFile(testFile)]
print(testTarget)
print(testData)

['money-fx']
["\n\nU.K. MONEY MARKET GIVEN 53 MLN STG ASSISTANCE\n\n    LONDON, April 8 - The Bank of England said it provided the\nmoney market with 53 mln stg assistance in the morning session.\n    This compares with the bank's estimate of a shortage in the\nsystem of around 300 mln stg which it earlier revised up from\n250 mln.\n    The central bank made outright purchases of bank bills\ncomprising 46 mln stg in band three at 9-3/4 pct and seven mln\nstg in band four at 9-11/16 pct.\n\n"]


In [67]:
X_new_counts = count_vect.transform(testData)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [68]:
predicted = clf.predict(X_new_tfidf)

In [69]:
for doc, category in zip(testTarget, predicted):
    print('%s => %s \n' % (doc, category))

money-fx => money-fx 



# Pipeline

In [70]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(analyzer = processDocument)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [71]:
categories = ["acq", "earn", "money-fx", "grain", "crude", "trade", "interest", "ship", "wheat", "corn"]

In [72]:
trainingFiles = getFilesInFolders("data/training", categories)
trainDocuments = [[filename.split("/")[2], readFile(filename)] for filename in concatFiles(trainingFiles)]
trainData = [content for file, content in trainDocuments]
trainTarget = [file for file, content in trainDocuments]

In [73]:
testFiles = getFilesInFolders("data/test", categories)
testDocuments = [[filename.split("/")[2], readFile(filename)] for filename in concatFiles(testFiles)]
testData = [content for file, content in testDocuments]
testTarget = [file for file, content in testDocuments]

In [74]:
text_clf.fit(trainData, trainTarget)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer=<function processDocument at 0x7fee3111cd90>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocesso...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [76]:
predicted = text_clf.predict(testData)

# Comparar resultados

In [75]:
import numpy as np

In [95]:
np.mean(predicted == testTarget)  

0.79332615715823462

In [90]:
same = np.where(predicted == testTarget)
different = np.where(predicted != testTarget)
print("same:", len(same[0]))
print("diff:", len(different[0]))
print(1-int(len(different[0]))/int(len(same[0])))

same: 2211
diff: 576
0.7394843962008142


In [96]:
print(different)

(array([   7,   10,   12,   21,   27,   34,   35,   40,   46,   56,   70,
         75,   80,   84,   87,  103,  105,  109,  113,  119,  123,  124,
        125,  127,  131,  132,  134,  136,  149,  151,  153,  155,  160,
        161,  162,  163,  165,  167,  168,  169,  173,  174,  175,  176,
        177,  182,  191,  192,  193,  201,  202,  203,  204,  205,  207,
        208,  210,  215,  216,  218,  220,  221,  223,  224,  225,  227,
        230,  233,  234,  236,  239,  242,  243,  245,  246,  250,  265,
        278,  281,  285,  287,  293,  295,  297,  298,  299,  300,  301,
        302,  303,  304,  305,  306,  308,  309,  315,  321,  325,  331,
        338,  342,  344,  346,  347,  348,  349,  350,  351,  353,  355,
        356,  357,  358,  360,  361,  363,  364,  365,  366,  367,  368,
        369,  370,  371,  372,  373,  374,  375,  376,  378,  382,  383,
        384,  385,  386,  387,  388,  389,  390,  391,  392,  393,  394,
        395,  396,  397,  399,  400,  401,  402,  

# Busqueda de diferencias

In [106]:
testSample = testFiles[7]
print(testSample)

['data/test/grain', '0012662']


In [107]:
testSampleTarget = [testSample[0].split("/")[2]]
testSampleData = [readFile("/".join(testSample))]
print(testSampleTarget)
print(testSampleData)

['grain']
['\n\nU.S. SENATE PANEL VOTES TO LIMIT COUNTY LOAN RATE  CHANGES STARTING WITH 1988 CROPS\n\nBlah blah blah.\n&#3;\n\n']


In [108]:
predicted = text_clf.predict(testSampleData)

In [109]:
for doc, category in zip(testTarget, predicted):
    print('%s => %s \n' % (doc, category))

grain => earn 



## Refactor

In [123]:
def evaluateTestSample(testFiles, index):
    testSample = testFiles[index]
    print("Sample")
    print(testSample)
    print()
    testSampleTarget = [testSample[0].split("/")[2]]
    testSampleData = [readFile("/".join(testSample))]
    print(testSampleData)
    print()
    print("Predicted:")
    predicted = text_clf.predict(testSampleData)
    for doc, category in zip(testTarget, predicted):
        print('%s => %s \n' % (doc, category))

In [124]:
evaluateTestSample(testFiles, 10)

Sample
['data/test/grain', '0011705']

['\n\nBANGLADESH NAVY UNLOADS GRAINS DURING PORT STRIKE\n\n    CHITTAGONG, Bangladesh, June 19 - The navy is unloading\nfoodgrains at Chittagong port following a strike by nearly\n1,000 dockworkers, Bangladesh Shipping Corp officials said.\n    The navy was unloading 74,000 tonnes of foodgrains from two\nships today, and four vessels laden with some 90,000 tonnes of\ngrains were waiting at the outer anchorage, port officials\nsaid.\n    A spokesman for the workers said they would continue their\nstrike until authorities agree to their demands for higher pay\nand other benefits. The strike began on June 11.\n    "Unloading of grains and other essential commodities started\nnormally after we called in navy personnel to help," a port\nofficial told Reuters.\n    "The strike has caused no serious dislocation," he said.\n    The government meanwhile declared the port jobs an\n"essential service" and said the strikers could be dismissed\nunless they end

In [125]:
evaluateTestSample(testFiles, 668)

Sample
['data/test/acq', '0009987']

["\n\nLEX SERVICE BUYS SEARS MOTOR GROUP\n\nc    LONDON, April 9 - Lex Service Plc <LEXL.L> said it had\nacquired <Sears Motor Group Ltd>, the retail motor distribution\narm of Sears Plc < SEHL.L>, and an 11.9 mln stg loan note\npayable by Sears Motor for 33.4 mln stg.\n    The purchase will be through 1.4 mln stg in cash and the\nissue to Sears Plc of 8.0 mln new Lex ordinary shares.\n    The company said in a statement that immediately following\nthe acquisition of the motor group, its car and commercial\nvehicle contract hire fleet of some 3,000 vehicles was sold to\n<Lex Vehicle Leasing Ltd> for 14.3 mln stg in cash, a sum equal\nto the net book value of the vehicles transferred.\n    Lex Vehicle is owned equally by Lex Services and <Lombard\nNorth Central Plc>.\n    Lex said the shares involved in the transaction were today\nbeing placed for Sears Plc with institutions at 400p. These\nshares will not qualify for the final Lex dividend on 10 Apr